## Getting to grips with the ACCESS-NRI Intake Catalog

There are three potential use cases for the ACCESS-NRI Intake Catalog:
1. You want to access some data someone else has *generated and catalogued*.
2. You want to *catalogue* some data you've generated.
3. You want to *access* some data you've catalogued.


Cases 2 & 3 can seem daunting - how do you build or access a custom catalog? This notebook should get you started, and will show you how to get to an ESM Datastore.

___
#### This notebook contains a basic example of how catalog an experiment & access the data in it using a simple wrapper function, and should get you up to speed with cataloging your experiments.
___

In [10]:
### Imports, make sure we are in our home directory (don't change this until you're sure you know why you want to)
!cd 
from pathlib import Path
import intake 
from access_nri_intake.source.builders import AccessOm2Builder, AccessOm3Builder, AccessEsm15Builder, AccessCm2Builder


## Case 1: Accessing an experiment in the standard catalog

In [9]:
# We've picked a demo experiment name here. You can find all available experiments in the standard catalog by calling `intake.cat.access_nri`.
experiment_name='01deg_jra55v13_ryf9091'

cat = intake.cat.access_nri
esm_ds = cat[experiment_name]
# Note: We call this esm_ds as this object is an `esm_datastore`: see https://intake-esm.readthedocs.io/en/v2021.8.17/user-guide/overview.html#loading-a-catalog for more details
esm_ds

,unique
path,11947
realm,2
variable,178
frequency,5
start_date,3361
end_date,3360
variable_long_name,181
variable_standard_name,36
variable_cell_methods,3
variable_units,50


In [7]:
# Now, if we wanted to subset and load some data, we could with, for example...
first_result = esm_ds.df.head(1).path
esm_ds.search(path=first_result).to_dask()

<xarray.Dataset> Size: 3GB
Dimensions:       (time: 1, d2: 2, nj: 2700, ni: 3600, nc: 5)
Coordinates:
  * time          (time) object 8B 1900-02-01 00:00:00
    TLON          (nj, ni) float32 39MB dask.array<chunksize=(675, 900), meta=np.ndarray>
    TLAT          (nj, ni) float32 39MB dask.array<chunksize=(675, 900), meta=np.ndarray>
    ULON          (nj, ni) float32 39MB dask.array<chunksize=(675, 900), meta=np.ndarray>
    ULAT          (nj, ni) float32 39MB dask.array<chunksize=(675, 900), meta=np.ndarray>
    NCAT          (nc) float32 20B dask.array<chunksize=(5,), meta=np.ndarray>
Dimensions without coordinates: d2, nj, ni, nc
Data variables: (12/49)
    time_bounds   (time, d2) object 16B dask.array<chunksize=(1, 2), meta=np.ndarray>
    tmask         (nj, ni) float32 39MB dask.array<chunksize=(675, 900), meta=np.ndarray>
    blkmask       (nj, ni) float32 39MB dask.array<chunksize=(675, 900), meta=np.ndarray>
    tarea         (nj, ni) float32 39MB dask.array<chunksize=(675, 900), meta=np.ndarray>
    uarea         (nj, ni) float32 39MB dask.array<chunksize=(675, 900), meta=np.ndarray>
    dxt           (nj, ni) float32 39MB dask.array<chunksize=(675, 900), meta=np.ndarray>
    ...            ...
    fmeltt_ai_m   (time, nj, ni) float32 39MB dask.array<chunksize=(1, 675, 900), meta=np.ndarray>
    opening_m     (time, nj, ni) float32 39MB dask.array<chunksize=(1, 675, 900), meta=np.ndarray>
    aicen_m       (time, nc, nj, ni) float32 194MB dask.array<chunksize=(1, 1, 675, 900), meta=np.ndarray>
    vicen_m       (time, nc, nj, ni) float32 194MB dask.array<chunksize=(1, 1, 675, 900), meta=np.ndarray>
    fmelttn_ai_m  (time, nc, nj, ni) float32 194MB dask.array<chunksize=(1, 1, 675, 900), meta=np.ndarray>
    flatn_ai_m    (time, nc, nj, ni) float32 194MB dask.array<chunksize=(1, 1, 675, 900), meta=np.ndarray>
Attributes: (12/24)
    title:                                    sea ice model output for CICE
    contents:                                 Diagnostic and Prognostic Varia...
    source:                                   Los Alamos Sea Ice Model (CICE)...
    comment:                                  This Year Has 365 days
    comment2:                                 File written on model date 1900...
    comment3:                                 seconds elapsed into model date...
    ...                                       ...
    intake_esm_attrs:variable_cell_methods:   ,,,,,,,,,,,,,,,,,,time: mean,ti...
    intake_esm_attrs:variable_units:          days since 1900-01-01 00:00:00,...
    intake_esm_attrs:filename:                iceh.1900-01.nc
    intake_esm_attrs:file_id:                 iceh_XXXX_XX
    intake_esm_attrs:_data_format_:           netcdf
    intake_esm_dataset_key:                   iceh_XXXX_XX.1mon

## Case 2: Generating a custom catalog for your experiment

We imported the builders above: pick from the one of these that matches the data you want to catalog. If you think you need a new builder, open an issue at https://github.com/ACCESS-NRI/access-nri-intake-catalog

Pick a builder from the following to set as `"builder"` below:
```python
from access_nri_intake.source.builders import AccessOm2Builder, AccessOm3Builder, AccessEsm15Builder, AccessCm2Builder
```
eg:
```python
catalog_config = {
    ...
    "builder" : AccessESM15Builder, 
    ...
}
```

In [11]:
# Set Config in this cell
catalog_config = {
      "experiment_output_path" : "/g/data/ik11/outputs/access-om2/1deg_iamip2_CMCC-ESM2ssp126", # Point this to your model output
      "builder" : AccessOm2Builder, # This must be one of the builders imported above. If you need a new builder, get in touch!
      "catalog_directory" : "./catalog_demo",
      "catalog_name" : "demo_datastore",
      "description" : "An example datastore for ACCESS-OM2 1deg_iamip2_CMCC-ESM2ssp126",
}

In [12]:
# This cell defines and runs a  function which will build and save our catalog.
def create_catalog(catalog_config):
    """
    Takes the `catalog_config` dictionary in the cell above and builds a catalog with the specified options
    """
    
    catalog_fname = Path(catalog_config["catalog_directory"]) / f"{catalog_config['catalog_name']}.json"

    catalog_written = catalog_fname.exists()
    
    parent_dir = catalog_fname.parent
    if not parent_dir.exists():
        parent_dir.mkdir(parents=True)

    if not catalog_written:
        print(f"No catalog found at specified output location: {catalog_fname}. Building catalog...")
        print("This could take a few minutes...")
        
        builder = AccessOm2Builder(
            path=catalog_config["experiment_output_path"]
        ).build()
        
        builder.save(
            name=catalog_config["catalog_name"],
            directory=catalog_config["catalog_directory"],
            description=catalog_config["description"],
        )
        print("Catalog built successfully!")
    else:
        print(f"Catalog alerady found at specified output location: {catalog_fname}. Did you mean to open it?")
        

# Now run it!
create_catalog(catalog_config)

No catalog found at specified output location: catalog_demo/demo_datastore.json. Building catalog...
This could take a few minutes...


/home/189/ct1163/access-nri-intake-catalog/src/access_nri_intake/source/utils.py:111: UserWarning: Time coordinate does not include bounds information. Guessing start and end times.
  warnings.warn(
/home/189/ct1163/access-nri-intake-catalog/src/access_nri_intake/source/utils.py:111: UserWarning: Time coordinate does not include bounds information. Guessing start and end times.
  warnings.warn(
/home/189/ct1163/access-nri-intake-catalog/src/access_nri_intake/source/utils.py:111: UserWarning: Time coordinate does not include bounds information. Guessing start and end times.
  warnings.warn(
/home/189/ct1163/access-nri-intake-catalog/src/access_nri_intake/source/utils.py:111: UserWarning: Time coordinate does not include bounds information. Guessing start and end times.
  warnings.warn(
/home/189/ct1163/access-nri-intake-catalog/src/access_nri_intake/source/utils.py:111: UserWarning: Time coordinate does not include bounds information. Guessing start and end times.
  warnings.warn(
/home

Successfully wrote ESM catalog json file to: file:///home/189/ct1163/catalog_demo/demo_datastore.json
Catalog built successfully!


## Case 3: Using a custom catalog

Once we've built a custom catalog, we can open it using the convenience function defined below - it uses the same configuration dictionary `catalog_config`, and so *nothing needs to be changed - just run the cell*. This will give us the same type of object that we got in __Case 1__ above, so from this point onwards, we can use it as if it were the standard catalog.


In [13]:
# This cell defines and runs a function which will open our catalog.
def open_catalog(catalog_config):
    """
    Looks for a catalog in the specified output location & attempts to open it
    """
    
    catalog_fname = Path(catalog_config["catalog_directory"]) / f"{catalog_config['catalog_name']}.json"

    if not catalog_fname.exists():
        print(f"No catalog found at specified output location: {catalog_fname}. Did you mean to build a catalog?")
    else:
        print(f"Catalog found at specified output location: {catalog_fname}. Opening catalog...")
        
    cat = intake.open_esm_datastore(
        catalog_fname,
        columns_with_iterables=["variable"] # We need to tell the datastore that the variable_column contains iterables or it won't work correctly
    )

    return cat

# Now run it!
custom_esm_ds = open_catalog(catalog_config)

Catalog found at specified output location: catalog_demo/demo_datastore.json. Opening catalog...


In [14]:
# Now, if we wanted to subset and load some data, we could with, for example...
first_result = custom_esm_ds.df.head(1).path
custom_esm_ds.search(path=first_result).to_dask()

<xarray.Dataset> Size: 2GB
Dimensions:      (time: 365, d2: 2, nj: 300, ni: 360, nc: 5)
Coordinates:
  * time         (time) datetime64[ns] 3kB 2015-01-02 2015-01-03 ... 2016-01-01
    TLON         (nj, ni) float32 432kB dask.array<chunksize=(150, 180), meta=np.ndarray>
    TLAT         (nj, ni) float32 432kB dask.array<chunksize=(150, 180), meta=np.ndarray>
    ULON         (nj, ni) float32 432kB dask.array<chunksize=(150, 180), meta=np.ndarray>
    ULAT         (nj, ni) float32 432kB dask.array<chunksize=(150, 180), meta=np.ndarray>
Dimensions without coordinates: d2, nj, ni, nc
Data variables: (12/24)
    time_bounds  (time, d2) datetime64[ns] 6kB dask.array<chunksize=(1, 2), meta=np.ndarray>
    NCAT         (nc) float32 20B dask.array<chunksize=(5,), meta=np.ndarray>
    tmask        (nj, ni) float32 432kB dask.array<chunksize=(150, 180), meta=np.ndarray>
    blkmask      (nj, ni) float32 432kB dask.array<chunksize=(150, 180), meta=np.ndarray>
    tarea        (nj, ni) float32 432kB dask.array<chunksize=(150, 180), meta=np.ndarray>
    dxt          (nj, ni) float32 432kB dask.array<chunksize=(150, 180), meta=np.ndarray>
    ...           ...
    algal_N      (time, nj, ni) float32 158MB dask.array<chunksize=(1, 150, 180), meta=np.ndarray>
    skl_Nit      (time, nj, ni) float32 158MB dask.array<chunksize=(1, 150, 180), meta=np.ndarray>
    ml_Nit       (time, nj, ni) float32 158MB dask.array<chunksize=(1, 150, 180), meta=np.ndarray>
    fNO_ai       (time, nj, ni) float32 158MB dask.array<chunksize=(1, 150, 180), meta=np.ndarray>
    fN_ai        (time, nj, ni) float32 158MB dask.array<chunksize=(1, 150, 180), meta=np.ndarray>
    PP_net       (time, nj, ni) float32 158MB dask.array<chunksize=(1, 150, 180), meta=np.ndarray>
Attributes: (12/27)
    title:                                    sea ice model output for CICE
    contents:                                 Diagnostic and Prognostic Varia...
    source:                                   Los Alamos Sea Ice Model (CICE)...
    time_period_freq:                         day_1
    comment:                                  This year has 365 days
    comment2:                                 File written on model date 2015...
    ...                                       ...
    intake_esm_attrs:variable_standard_name:  ['', '', '', '', '', '', '', ''...
    intake_esm_attrs:variable_cell_methods:   ['', '', '', '', '', '', '', ''...
    intake_esm_attrs:variable_units:          ['days since 2015-01-01 00:00:0...
    intake_esm_attrs:realm:                   seaIce
    intake_esm_attrs:_data_format_:           netcdf
    intake_esm_dataset_key:                   iceh_XXX_daily.1day